In [1]:
import os
from distutils.log import debug
from fileinput import filename
from flask import *  
from werkzeug.utils import secure_filename
from openpyxl import load_workbook
import pandas as pd
import win32com.client as win32 
import pythoncom
import pyodbc
import openpyxl
import numpy as np
import warnings
from openpyxl.styles import Font
warnings.filterwarnings('ignore')
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
from openpyxl.styles.colors import Color
from openpyxl.drawing.image import Image
import excel2img
import shutil
#import temp_folder
import tempfile
import time
import sys

C:\Users\nasreenn\AppData\Local\Temp\ipykernel_1020\2830447149.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.log import debug


In [2]:
DB = {'servername': 'aW02PSQLC007',
    'database': 'India_GTSG'}
conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
rft_report = pd.read_sql_query('''SELECT * FROM [India_GTSG].[dbo].[RFT_P6]''', conn)
ut = pd.read_sql_query('''SELECT * FROM [India_GTSG].[dbo].[UT]''', conn)
calendar = pd.read_sql_query('''SELECT * FROM [India_GTSG].[dbo].[calendar]''', conn)
data = pd.read_excel('275222.xlsx', sheet_name = 'P6')
rft_report = rft_report.drop(['Month-Year', 'Utilization_Target_percent'], axis = 1)

In [3]:
 # Popping the column to change the position
cll1 = rft_report.pop('Utilization_Target')
cll2 = rft_report.pop('Employee_code')
cll3 = rft_report.pop('Resource_Name')
cll4 = rft_report.pop('Project Number')

    # Placing the pop up column at first
rft_report.insert(2, 'Utilization_Target', cll1)
# Placing the pop up column at first
rft_report.insert(2, 'Employee_code', cll2)
    # Placing the pop up column at first
rft_report.insert(2, 'Resource_Name', cll3)
    # Placing the pop up column at first
rft_report.insert(2, 'Project Number', cll4)

    # To replace dash (-) & NaN with Zero (0)
except_columns = ['Project', 'Resource (Utilization Target %)/Job Code', 'Project Number', 'Resource_Name', 'Employee_code', 'Utilization_Target']
for column in rft_report.columns:
    if column not in except_columns:
        rft_report[column] = rft_report[column].replace('-', 0)
        rft_report[column] = rft_report[column].replace(np.NaN, 0)

    # rft_report.isnull().sum()
    # To change the dtypes of columns
except_columns = ['Project', 'Resource (Utilization Target %)/Job Code', 'Project Number', 'Resource_Name', 'Employee_code', 'Utilization_Target']
for column in rft_report.columns:
    if column not in except_columns:
        rft_report[column] = pd.to_numeric(rft_report[column], errors = 'coerce')    

In [4]:
 # Dropping unwanted columns
data1 = data.drop(['Activity Name', 'Curve', 'Spreadsheet Field', 'Remaining Units'], axis = 1)

In [5]:
#changing the format of date and adding it in a new column
calendar['Month_Year'] = calendar['GL Period'].apply(lambda x: x.strftime('%b-%y')) 

In [6]:
# Extracting Month columns 
data2 = data1.iloc[:, 1:]
data2

,2022-05-29,2022-07-03,2022-07-31,2022-08-28,2022-10-02,2022-10-30,2022-11-27,2023-01-01,2023-01-29,2023-02-26,...,2026-11-22,2026-12-27,2027-01-24,2027-02-21,2027-03-28,2027-04-25,2027-05-23,2027-06-27,2027-07-25,2027-08-22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from datetime import datetime
# data2.columns = pd.DataFrame(data2.columns)

In [8]:
calendar = calendar[calendar['W/E Date'].isin(data2.columns)]

In [9]:
# Get the current date
import datetime
current_date = datetime.datetime.now()
filtered_df = calendar[calendar['W/E Date'] <current_date]
filtered_df['W/E Date'] = filtered_df['W/E Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
col = filtered_df.tail(1)
date = col['W/E Date'].tolist()
dt = date[0]
cal = calendar[calendar['W/E Date'] >= dt]
cal = cal.reset_index(drop = True)
final_cal = cal[0:18]
final_cal

,W/E Date,Week No.,GL Period,Month_Year
0,2023-10-01,40,2023-10-01,Oct-23
1,2023-10-29,44,2023-11-01,Nov-23
2,2023-11-26,48,2023-12-01,Dec-23
3,2023-12-31,1,2024-01-01,Jan-24
4,2024-01-28,5,2024-02-01,Feb-24
5,2024-02-25,9,2024-03-01,Mar-24
6,2024-03-31,14,2024-04-01,Apr-24
7,2024-04-28,18,2024-05-01,May-24
8,2024-05-26,22,2024-06-01,Jun-24
9,2024-06-30,27,2024-07-01,Jul-24


In [17]:
data2

,2022-05-29,2022-07-03,2022-07-31,2022-08-28,2022-10-02,2022-10-30,2022-11-27,2023-01-01,2023-01-29,2023-02-26,...,2026-11-22,2026-12-27,2027-01-24,2027-02-21,2027-03-28,2027-04-25,2027-05-23,2027-06-27,2027-07-25,2027-08-22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
final_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   W/E Date    18 non-null     datetime64[ns]
 1   Week No.    18 non-null     int64         
 2   GL Period   18 non-null     datetime64[ns]
 3   Month_Year  18 non-null     object        
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 704.0+ bytes


In [12]:
final_cal['W/E Date'] = final_cal['W/E Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [13]:
final_cal

,W/E Date,Week No.,GL Period,Month_Year
0,2023-10-01 00:00:00,40,2023-10-01,Oct-23
1,2023-10-29 00:00:00,44,2023-11-01,Nov-23
2,2023-11-26 00:00:00,48,2023-12-01,Dec-23
3,2023-12-31 00:00:00,1,2024-01-01,Jan-24
4,2024-01-28 00:00:00,5,2024-02-01,Feb-24
5,2024-02-25 00:00:00,9,2024-03-01,Mar-24
6,2024-03-31 00:00:00,14,2024-04-01,Apr-24
7,2024-04-28 00:00:00,18,2024-05-01,May-24
8,2024-05-26 00:00:00,22,2024-06-01,Jun-24
9,2024-06-30 00:00:00,27,2024-07-01,Jul-24


In [14]:
# # Sample array
# # array = ["2023-10-15 00:00:00", "2023-11-20 12:30:00", "2024-01-05 08:00:00"]

# # Replace "00:00:00" with an empty string in each element
# # months = [element.replace(" 00:00:00", "") for element in month]
# # months
# # Convert the array to a list
# my_list = list(month)

# # Print the resulting list
# print(my_list)

In [15]:
# #extracting all amonth-year values
# month = final_cal['W/E Date'].values
# data2 = data2[data2.columns.isin(months)]
# data5 = data2.set_axis(month, axis = 'columns')
# data5

In [16]:
    # Joining df with p_six
    data5.insert(loc = 0, column = 'Activity ID', value = data1['Activity ID'])

    # Extracting rows with 'Resource Name'
    names = data5[data5['Activity ID'].str.contains("Resource")]

    # Creating new column named 'Resource Name' by replacing Resource name from col 'Activity ID'
    names['Resource Name'] = names['Activity ID'].str.replace('Resource Name: ', '')

    # Dropping unwanted columns
    dataset = names.drop('Activity ID', axis = 1)

    # Popping the column to change the position
    val1 = dataset.pop('Resource Name')

    # Placing the pop up column at first
    dataset.insert(0, 'Resource Name', val1)

    # Reseting Index
    data6 = dataset.reset_index()

    # Dropping unwanted columns
    data7 = data6.drop(['index'], axis = 1)

    # Changing column name '0' ---> 'Project Number'
    data7.rename(columns = {'Resource Name': 'Resource_Name'}, inplace = True)

    # Remove extra space
    data7['Resource_Name'] = data7['Resource_Name'].apply(str.strip)

    # Extracting rows with 'WBS'
    wbs = data5[data5['Activity ID'].str.contains("WBS: ")]

    # Extracting numbers from main column
    wbs['num'] = wbs['Activity ID'].str.findall(r'[0-9]+')

NameError: name 'data5' is not defined

In [20]:
from datetime import datetime

# Assuming you have a timestamp as a datetime object
timestamp = datetime(2022, 5, 29, 0, 0, 0)

# Convert the timestamp to a string
timestamp_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')

# Extract the text within the timestamp
text_within_timestamp = timestamp_str

print(text_within_timestamp)

2022-05-29 00:00:00


In [24]:
timestamp_str

'2022-05-29 00:00:00'

In [26]:
from pandas import Timestamp
x = Timestamp('2022-05-29 00:00:00')

In [30]:
x2 = x.strftime('%Y-%m-%d %H:%M:%S')

In [31]:
x2

'2022-05-29 00:00:00'